# Simple Expression Parser
Implementation of a simple mathematical expression parser. This implements a Stack class which extends a Linkedist, to perform postfix evaluation (1 2 + = ?), then infix evaluation (1 + 2 = ?).

In [1]:
class Node():
    def __init__(self,data):
        self.data = data
        self.next = None
        self.prev = None

In [2]:
class LinkedList():
    
    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0
        
    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1
        
    def __iter__(self):
        self._iter_node = self.head
        return self 
    
    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret
    
    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        self.length += 1
        
    def __len__(self):
        return self.length
    
    # Add __str__() method here
    def __str__(self):
        py_list = [x for x in self]
        return str(py_list)

In [3]:
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
    def peek(self):
        return self.tail.data
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

In [4]:
# First a tokenizer - assume there will be spaces
def tokenizer(token_str):
    return (token_str.split())

In [5]:
expression = "12 2 4 + / 21 *"
print(tokenizer(expression))

['12', '2', '4', '+', '/', '21', '*']


In [6]:
# Add processing for each operator.
# These all implement an infix expression where the right side is stack[top] and left side is stack[second from top]
# A minus function:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)
        
# A plus function:
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

# A times function:
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)
        
# A divide function:
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)
        
# A power function:
def process_power(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

In [7]:
stack = Stack()
for sign, func in ('-', process_minus), ('+', process_plus), ('*', process_times), \
                  ('/', process_divide), ('**', process_power):
    for x,y in (1,2), (5,5), (0, 10):
        print (x, sign, y, '=', end=' ')
        stack.push (x); stack.push (y)
        func(stack)
        print (stack.pop())

1 - 2 = -1
5 - 5 = 0
0 - 10 = -10
1 + 2 = 3
5 + 5 = 10
0 + 10 = 10
1 * 2 = 2
5 * 5 = 25
0 * 10 = 0
1 / 2 = 0.5
5 / 5 = 1.0
0 / 10 = 0.0
1 ** 2 = 1
5 ** 5 = 3125
0 ** 10 = 0


In [8]:
expression = "12 2 4 + / 21 *"



In [9]:
def evaluate_postfix(expression):
    process_funcs = {'-':process_minus,  \
                     '+':process_plus,   \
                     '*':process_times,  \
                     '/':process_divide, \
                     '**':process_power}

    tokens = tokenizer(expression)
    operating_stack = Stack()
    result = None

    for token in tokens:
        if token in process_funcs:
            process_funcs[token](operating_stack)
        else:
            operating_stack.push(float(token))
    return operating_stack.pop()

In [10]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]
for expr in expressions:
    print (expr, '->', evaluate_postfix(expr))

4 6 - -> -2.0
4 1 2 9 3 / * + 5 - * -> 8.0
1 2 + 3 - -> 0.0
1 2 - 3 + -> 2.0
10 3 5 * 16 4 - / + -> 11.25
5 3 4 2 - ** * -> 45.0
12 2 4 + / 21 * -> 42.0
1 1 + 2 ** -> 4.0
1 1 2 ** + -> 2.0


## 'Infix' evaluation
We'll now implement a ['Shunting-yard' algorithm](https://en.wikipedia.org/wiki/Shunting-yard_algorithm) to convert from an infix to a postfix expression. For example from: ```4 - 6``` to ```4 6 -```.

In [11]:
# Set up precedence of operators:
precedence = {'+':2, '-':2, '*':3, '/':3, '**':4, '^':4}
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

True
False
True


Here's how processing should go for each token:

Opening parenthesis, (:
- Push the token into the stack for later use when we find a closing parenthesis.

Closing parenthesis ):
- While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list.
- Pop the opening parentheses out of the stack at the end.

Operator, +, -, \*, /, or \**
- While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list.
- Push the current operator to the top of the stack.

Operand (any number):
- Append the number to the postfix token list.

After having processed all tokens, the stack may not be empty, so then pop any remaining operators into the postfix list.

In [12]:
# Open bracket function:
def process_open_bracket(stack):
    stack.push("(")
    
# Close bracket function:
def process_close_bracket(stack, postfix):
    # Pop everything off the stack onto the postfix token list, until we get to "("
    while stack.peek() != "(":
        postfix.append(stack.pop())
    # Now there should be a "(" left on the stack to throw away
    stack.pop()
    
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and \
          precedence[stack.peek()] >= precedence[operator]:
        # Pop the top of the stack into the postfix list
        postfix.append(stack.pop())
    stack.push(operator)
    
def process_number(postfix, number):
    postfix.append(number)

In [13]:
def infix_to_postfix(infix):
    stack = Stack()
    postfix = []
    for token in tokenizer(infix):
        if token == "(":
            process_open_bracket(stack)
        elif token == ")":
            process_close_bracket(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            # Assume number
            process_number(postfix, token)
        
    while len(stack):
        postfix.append(stack.pop())
    
    return " ".join(postfix)

In [14]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)


In [15]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expr in expressions:
    print(expr, "->", evaluate(expr))

1 + 1 -> 2.0
1 * ( 2 - ( 1 + 1 ) ) -> 0.0
4 * ( 1 + 2 * ( 9 / 3 ) - 5 ) -> 8.0
10 + 3 * 5 / ( 16 - 4 * 1 ) -> 11.25
2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 -> 256.0
2 ** 2 ** 2 ** 2 ** 2 -> 65536.0
( 1 - 2 ) / ( 3 - 5 ) -> 0.5
9 / 8 * 8 -> 9.0
64 / ( 8 * 8 ) -> 1.0


In [16]:
evaluate("1 / 3")

0.3333333333333333

## To do next
One of the limitations of this implementation is that it requires spaces to separate every part of the expression. For example, it can evaluate the expression 2 * ( 5 - 3 ), but it can't evaluate 2 * (5 - 3) or 2*(5 - 3).

Also, implement other operators — for example, the integer division //.

The evaluate() function that we implemented exists in Python as the eval() built-in function. The Python implementation can actually evaluate any string of Python code, not only expressions.

In [17]:
eval("print('Hello')")

Hello


In [18]:
eval ("1/3")

0.3333333333333333